# Библиотеки

In [6]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from html.parser import HTMLParser
from numpy import array as Arr
import pandas as pd
import os, io, re
import regex

from matplotlib.patches import Circle, RegularPolygon
from matplotlib.projections import register_projection
from matplotlib.projections.polar import PolarAxes
from matplotlib.transforms import Affine2D
from matplotlib.ticker import FixedLocator
from matplotlib.spines import Spine
from matplotlib.path import Path
from math import pi

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np

from docxtpl import DocxTemplate, InlineImage, RichText
from bs4 import BeautifulSoup as BS
from docx.shared import Mm, Cm
import PySimpleGUI as sg
import datetime
import docx
import urllib.request
pd.options.mode.chained_assignment = None

import warnings
warnings.filterwarnings('ignore')

# Авторизация + скачивание

In [7]:
def auth():
    gauth = GoogleAuth()
    gauth.LoadCredentialsFile("mycreds.txt")
    if gauth.credentials is None:
        gauth.LocalWebserverAuth()
    elif gauth.access_token_expired:
        gauth.Refresh()
    else:
        gauth.Authorize()
    gauth.SaveCredentialsFile("mycreds.txt")
    drive = GoogleDrive(gauth)
    return(drive)

def transliterate(string): # Транслитерация
    let_dict = {u'а': u'a', u'б': u'b', u'в': u'v', u'г': u'g', u'д': u'd', u'е': u'e', u'ё': u'e', u'ж': u'zh', u'з': u'z',
                    u'и': u'i', u'й': u'y', u'к': u'k', u'л': u'l', u'м': u'm', u'н': u'n', u'о': u'o', u'п': u'p', u'р': u'r',
                    u'с': u's', u'т': u't', u'у': u'u', u'ф': u'f', u'х': u'h', u'ц': u'ts', u'ч': u'ch', u'ш': u'sh',
                    u'щ': u'sch', u'ъ': u'', u'ы': u'y', u'ь': u'', u'э': u'e', u'ю': u'yu', u'я': u'ya',} 
    result = u""
    for index, char in enumerate(string.lower()):
        if char in let_dict:
            char = let_dict[char]
        result += char
    return result.title()

def search(names, name1, name2, trans = False): # Поиск по Ф и И
    if trans:
        r = regex.compile(r"(%s){e<=1}" % transliterate(name1.lower()))
        r1 = regex.compile(r"(%s){e<=1}" % transliterate(name2.lower()))
        result = [name for name in names if r.findall(transliterate(name.lower())) and r1.findall(transliterate(name.lower()))]
    else:
        r = regex.compile(r"(%s){e<=1}" % name1.lower())
        r1 = regex.compile(r"(%s){e<=1}" % name2.lower())
        result = [name for name in names if r.findall(name.lower()) and r1.findall(name.lower())]
    return result

def get_docs(drive):
    file_list = drive.ListFile({'q': "title contains '_игрок_' and title contains '.html'"}).GetList()
    files = {'name': [], 'id': [], 'parent_id': []}
    for file in file_list:
        files['name'].append(file['title'])
        files['id'].append(file['id'])
        files['parent_id'].append(file['parents'][0]['id'])
    docs = pd.DataFrame(files, dtype = str)
    return(docs)

def game_name(parent_id):
    parent_id_now = parent_id
    while parent_id_now != root_id:
        id_now = parent_id_now
        file_now = drive.CreateFile({'id': id_now})
        parent_id_now = file_now['parents'][0]['id']
    return(file_now['title'])

def get_nice(drive):
    file = drive.ListFile({'q': "title = 'Nice2MeetYou' "}).GetList()[0]
    file.GetContentFile(file['title']+'.xlsx', mimetype = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    nice = pd.read_excel(file['title']+'.xlsx', dtype = str, keep_default_na = False)
    return(nice)

def get_db_list(drive):
    file = drive.ListFile({'q': "title = 'Список файлов БД' "}).GetList()[0]
    file.GetContentFile(file['title']+'.xlsx', mimetype = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    db_list = pd.read_excel(file['title']+'.xlsx', dtype = str, keep_default_na = False)
    return(db_list)

# Гиперпицца

In [39]:
take_money_list, comp_cr2_dict = [], {}
emp_hr_count2_dict, make_deal_dict = {}, {}
emp_hr_total_dict, emp_cost_max = {}, {}
emp_hr_count3_dict, emp_hr_dict = {}, {}
comp_cr3_dict, comp_cr4_dict = {}, {}
emp_hr_count4_dict, buy_rnd4_list = {}, []

## Preprocess text
class MyHTMLParserHP(HTMLParser):
    def handle_data(self, data):
        if not re.match(r'^\s*$', data):
            result.append(data)

## Get table and result list
def table_result_score(filepath):
    global result
    table_new, result = [], []

    parser = MyHTMLParserHP()
    with io.open(filepath, encoding = 'utf-8') as file:
        for line in file:
            parser.feed(line)
    
    global player_name
    #player_name = [line for line in result if line.startswith('Игрок')][0][7:]
    score = float([line for line in result if line.startswith('Баланс')][0][8:])
    table = result[result.index('Таблица транзакций') + 3 : result.index('Описание транзакций')]
    for i in range(0, len(table), 2):
        table_new.append((table[i], table[i + 1]))

    del result[:result.index('Описание транзакций') + 1]
    levels = [i for i, x in enumerate(result) if x[-2:] == ': ']
    #result = [result[i].lower() for i in range(len(result))]
    result = [result[i].lower() + result[i + 1].lower() if i in levels else result[i] for i in range(len(result))]
    result = [result[i] for i in range(len(result)) if i - 1 not in levels]
    indexs = [i for i, x in enumerate(result) if x == 'выполнена' or x == 'отклонена']
    indexs.append(len(result))
    result = [result[indexs[n]:indexs[n + 1]] for n in range(len(indexs)-1)]
    result = [x if 'уровень игры: ' in ''.join(x) else \
            x.append('уровень игры: ' + table[table.index(x[1])+1]) or x for x in result]
    #result = [x[:x.index('отклонена')] if 'отклонена' in x else x for x in result]
    result = [x[:x.index('ожидает одобрения участников')] if 'ожидает одобрения участников' in x else x for x in result]
    return(table_new, result, score)

## Rules Function
def rules(code, filename, marks):
    numbers = Arr([0]*8)
    if code == "1.1":
        if ('зафиксировать инженерам распределение приза от корпорации X', '1') in tables[filename]:
            take_money_list.append(filename)
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "1.2":
        if ('подарить талеры', '1') in tables[filename] \
                and any([y[y.find('кто')+5:] in ' '.join(take_money_list).lower() \
                for y in sum([x for x in results[filename] if 'подарить талеры' in x \
                and 'уровень игры: 1' in x], []) if y.startswith('кто')]):
            numbers += Arr([0,1,0,0,0,0,0,0])
    elif code == "1.3":
        if ('взять у скаута инструмент в аренду', '1') in tables[filename] \
                and any([y for y in sum([x for x in results[filename] if 'взять у скаута инструмент в аренду' in x \
                and 'уровень игры: 1' in x], []) if y.startswith('cкаут')]): # english c
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "1.4":
        if ('покупка готовой продукции', '2') in tables[filename] or \
                any([y for y in sum([x for x in results[filename] if 'взять у скаута инструмент в аренду' in x \
                and 'уровень игры: 1' in x], []) if y.startswith('инженер')]):
            numbers += Arr([2,0,0,0,0,0,0,0])
    elif code == "1.5":
        if filename in ' '.join(take_money_list) \
                and all([('подарить талеры', k) not in tables[filename] for k in ['1', '2']]):
            numbers += Arr([0,0,1,1,0,0,0,0])
    elif code == "1.6":
        if filename in ' '.join(take_money_list) \
                and any([('подарить талеры', k) in tables[filename] for k in ['1', '2']]):
            numbers += Arr([0,0,0,0,0,1,0,0])
    elif code == "2.0":
        for comp_name in [y[y.find('название')+10:] \
                for y in sum([x for x in results[filename] if 'создать компанию' in x \
                and 'уровень игры: 2' in x], []) if y.startswith('название')]:
            comp_cr2_dict[comp_name].append(filename) if comp_name in comp_cr2_dict.keys() \
                    else comp_cr2_dict.update({comp_name: [filename]})
    elif code == "2.1":
        if all([(action, '2') in tables[filename] for action in \
                ['создать компанию', 'перевести талеры на счет компании', 'нанять сотрудника']]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "2.2":
        if ('перевести талеры на счет компании', '2') in tables[filename] \
                and max([len(x) for x in comp_cr2_dict.values() if filename in x]) > 1:
            numbers += Arr([0,0,1,0,0,1,0,0])
    elif code == "2.3":
        if any([y for y in sum([x for x in results[filename] if 'тип договора: продажа инструмента' in x \
                and 'уровень игры: 2' in x], [])]): # english c
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "2.4":
        if scores[filename] <= -400 and tables[filename].count(('нанять сотрудника', '2')) < 5:
            numbers += Arr([-1,0,-1,0,0,0,0,0])
    elif code == "2.5":
        if scores[filename] <= -400 and tables[filename].count(('нанять сотрудника', '2')) >= 5:
            numbers += Arr([-1,0,-1,0,0,1,0,0])
    elif code == "2.6":
        emp_hr_count2_dict[filename] = len([y[y.find('ceo')+5:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 2' in x], []) if y.startswith('ceo')]) - \
                len([y for y in sum([x for x in results[filename] if 'уволить сотрудника' in x \
                and 'уровень игры: 2' in x], [])])
        emp_cost_max[filename[7:-5].lower()] = max([int(y[y.find('зарплата')+10:]) \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x and 'уровень игры: 2' in x \
                and any(['инженер: '+filename[7:-5].lower() in z for z in x])], []) if y.startswith('зарплата')], default=0)
        if emp_hr_count2_dict[filename] > 3:
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "2.7":
        if ('покупка готовой продукции', '2') in tables[filename] \
                and len([x for x in results[filename] if 'от компании получен  продукт(услуга): отдел закупок корпорации 1'
                in x or 'от компании получен  продукт(услуга): отдел закупок корпорации 2' in x]):
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "2.8":
        if any([filename in x for x in comp_cr2_dict.values()]) \
                and max([len(x) for x in comp_cr2_dict.values() if filename in x]) > 1 \
                and ('перевести талеры на счет компании', '2') not in tables[filename]:
            numbers += Arr([0,0,-2,0,0,0,1,0])
    elif code == "2.9":
        if any([y[y.find('ceo')+5:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and 'уровень игры: 2' in x], []) if y.startswith('ceo')]):
            numbers += Arr([1,0,0,1,0,0,0,0])
        make_deal_dict[filename] = list(set([y[y.find('тип договора')+14:] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x and 'уровень игры: 2' in x \
                and all([filename[7:-5].lower() not in z for z in x])], []) if y.startswith('тип договора')]))
    elif code == "3.0":
        for comp_name in [y[y.find('название')+10:] \
                for y in sum([x for x in results[filename] if 'создать компанию' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('название')]:
            comp_cr3_dict[comp_name].append(filename) if comp_name in comp_cr3_dict.keys() \
                    else comp_cr3_dict.update({comp_name: [filename]})
    elif code == "3.1":
        if any([y[y.find('инвестор')+10:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'инвестировать' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('инвестор')]):
            numbers += Arr([0,0,0,2,0,0,0,0])
    elif code == "3.2":
        make_deal_x = ([y[y.find('тип договора')+14:] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x and 'уровень игры: 3' in x \
                and all([filename[7:-5].lower() not in z for z in x])], []) if y.startswith('тип договора')])
        if any([y not in make_deal_dict[filename]+['материалы'] for y in make_deal_x]) or make_deal_x.count('материалы') > 2:
            numbers += Arr([1,0,0,2,3,0,0,0])
    elif code == "3.3":
        if ('нанять сотрудника', '3') in tables[filename]:
            numbers += Arr([0,0,0,0,0,0,0,0])
    elif code == "3.4":
        emp_hr_count3_dict[filename] = len([y[y.find('ceo')+5:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('ceo')]) - \
                len([y for y in sum([x for x in results[filename] if 'уволить сотрудника' in x \
                and 'уровень игры: 3' in x], [])])
        if emp_hr_count3_dict[filename] < min(3, emp_hr_count2_dict[filename]):
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "3.5":
        comp_x = [comp for comp, names in list(comp_cr2_dict.items()) + list(comp_cr3_dict.items()) + list(comp_cr4_dict.items()) \
                          if filename in names]
        balances = []
        for comp in comp_x[:]:
            balance_x = 0
            index_max_x = max([results[filename].index(x) for x in results[filename] if 'нанять сотрудника' in x \
                            and 'отклонена' in x and 'уровень игры: 4' not in x and 'компания: '+comp in x], default=0)
            for x in results[filename][:index_max_x+1]:
                if 'выполнена' in x and 'компани' in ' '.join(x) and comp in ' '.join(x):
                    if 'перевести талеры на счет компании' in x and 'собственник: '+filename[7:-5].lower() in x:
                        cost_raw = [y for y in x if y.startswith('взнос')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'нанять сотрудника' in x and 'ceo: '+filename[7:-5].lower():
                        cost_raw = [y for y in x if y.startswith('зарплата')][0]
                        balance_x -= int(cost_raw[cost_raw.find(' ')+1 :])
                    elif 'покупка готовой продукции' in x or 'заключить сделки' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        num_raw = [y for y in x if y.startswith('кол-во')][0]
                        if 'получатель компания: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                        elif 'ceo: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                    elif 'купить продукт B2C' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        if 'у компании: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                        elif 'покупатель: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'инвестировать' in x:
                        cost_raw = [y for y in x if y.startswith('инвестиции')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
            balances.append(balance_x)
        if max(balances, default=0) >= 50 and emp_hr_count3_dict[filename] < min(3, emp_hr_count2_dict[filename]):
            numbers += Arr([0,0,-1,0,0,0,0,0])
    elif code == "3.6":
        comp_x = [comp for comp, names in list(comp_cr2_dict.items()) + list(comp_cr3_dict.items()) + list(comp_cr4_dict.items()) \
                          if filename in names]
        balances = []
        for comp in comp_x[:]:
            balance_x = 0
            index_max_x = max([results[filename].index(x) for x in results[filename] if 'нанять сотрудника' in x \
                            and 'уровень игры: 4' not in x and 'компания: '+comp in x], default=0)
            for x in results[filename][:index_max_x+1]:
                if 'выполнена' in x and 'компани' in ' '.join(x) and comp in ' '.join(x):
                    if 'перевести талеры на счет компании' in x and 'собственник: '+filename[7:-5].lower() in x:
                        cost_raw = [y for y in x if y.startswith('взнос')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'нанять сотрудника' in x and 'ceo: '+filename[7:-5].lower():
                        cost_raw = [y for y in x if y.startswith('зарплата')][0]
                        balance_x -= int(cost_raw[cost_raw.find(' ')+1 :])
                    elif 'покупка готовой продукции' in x or 'заключить сделки' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        num_raw = [y for y in x if y.startswith('кол-во')][0]
                        if 'получатель компания: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                        elif 'ceo: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                    elif 'купить продукт B2C' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        if 'у компании: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                        elif 'покупатель: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'инвестировать' in x:
                        cost_raw = [y for y in x if y.startswith('инвестиции')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
            balances.append(balance_x)
        if max(balances, default=0) >= 50 and emp_hr_count3_dict[filename] < min(3, emp_hr_count2_dict[filename]):
            numbers += Arr([0,1,0,0,0,0,0,0])
    elif code == "3.7":
        if any([y[y.find('ceo')+5:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and all(['от компании получен  продукт(услуга): компания х1' not in x, \
                         'от компании получен  продукт(услуга): компания х2' not in x]) \
                and 'уровень игры: 3' in x], []) if y.startswith('ceo')]):
            numbers += Arr([1,1,1,0,0,0,0,0])
    elif code == "3.8":
        make_deal_dict[filename] += ([y[y.find('тип договора')+14:] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x and 'уровень игры: 3' in x \
                and all([filename[7:-5].lower() not in z for z in x])], []) if y.startswith('тип договора')])
        if len(make_deal_dict[filename]) != len(set(make_deal_dict[filename])):
            numbers += Arr([0,0,1,1,0,0,0,0])
        make_deal_dict[filename] = list(set(make_deal_dict[filename]))
    elif code == "3.9":
        emp_hr_total_dict[filename] = [y[y.find('инженер')+9:] \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and any(['ceo: '+filename[7:-5].lower() in z for z in x])], []) if y.startswith('инженер')]
        if sum([y[y.find('покупатель')+12:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'купить продукт B2C' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('покупатель')]) >= 2:
            numbers += Arr([3,0,0,0,3,0,0,0])
    elif code == "3.10":
        if len([y[y.find('инвестор')+10:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'инвестировать' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('инвестор')]) > 1:
            numbers += Arr([0,0,1,0,1,0,0,0])
    elif code == "3.11":
        if any([y[y.find('ceo')+5:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and any(['от компании получен  продукт(услуга): компания х1' in x, \
                         'от компании получен  продукт(услуга): компания х2' in x]) \
                and 'уровень игры: 3' in x], []) if y.startswith('ceo')]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "3.12":
        emp_hr_dict[filename] = [y[y.find('инженер')+9:] \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x and 'уровень игры: 2' in x \
                and any(['ceo: '+filename[7:-5].lower() in z for z in x])], []) if y.startswith('инженер')]
        emp_hr_x = [dict([xx.split(': ') for xx in x if ':' in xx]) for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 3' in x and any(['ceo: '+filename[7:-5].lower() in z for z in x]) \
                and all(['инженер: '+emp not in x for emp in emp_hr_dict[filename]])]
        if any([int(_dict['зарплата']) > emp_cost_max[_dict['инженер']] for _dict in emp_hr_x]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "3.13":
        emp_hr_x = [dict([xx.split(': ') for xx in x if ':' in xx]) for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 3' in x and any(['ceo: '+filename[7:-5].lower() in z for z in x]) \
                and any(['инженер: '+emp in x for emp in emp_hr_dict[filename]])]
        if any([int(_dict['зарплата']) > emp_cost_max[_dict['инженер']] for _dict in emp_hr_x]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "3.14":
        revenue_x = [dict([xx.split(': ') for xx in x if ':' in xx]) for x in results[filename] if 'заключить сделки' in x \
            and 'уровень игры: 3' in x and any(['ceo: '+filename[7:-5].lower() in z for z in x])]
        if emp_hr_count3_dict[filename] > 3 and all([(float(x['цена'])*int(x['кол-во'])) < 10 for x in revenue_x]):
            numbers += Arr([0,0,0,0,0,3,0,0])
    elif code == "4.0":
        if any([y[y.find('инвестор')+10:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'инвестировать' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('инвестор')]): #3.1 copy
            numbers += Arr([0,0,2,2,0,0,0,0])
        if len([y[y.find('инвестор')+10:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'инвестировать' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('инвестор')]) > 1: #3.10 copy
            numbers += Arr([0,0,1,0,1,0,0,0])
        for comp_name in [y[y.find('название')+10:] \
                for y in sum([x for x in results[filename] if 'создать компанию' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('название')]:
            comp_cr4_dict[comp_name].append(filename) if comp_name in comp_cr4_dict.keys() \
                    else comp_cr4_dict.update({comp_name: [filename]})
        for comp_name in [y[y.find('получатель компания')+21:] \
                for y in sum([x for x in results[filename] if 'покупка готовой продукции' in x \
                and 'уровень игры: 4' in x and 'от компании получен  продукт(услуга): отдел rnd корпорации х' in x], []) \
                if y.startswith('получатель компания')]:
            buy_rnd4_list.append(comp_name)
    elif code == "4.1":
        if ('закрыть компанию', '4') in tables[filename] and \
                max([len(x) for x in comp_cr4_dict.values() if filename in x], default=0) == 1:
            numbers += Arr([0,0,1,0,0,1,0,0])
    elif code == "4.2":
        make_deal_dict[filename] += ([y[y.find('тип договора')+14:] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x and 'уровень игры: 4' in x \
                and all([filename[7:-5].lower() not in z for z in x])], []) if y.startswith('тип договора')])
        if len(make_deal_dict[filename]) != len(set(make_deal_dict[filename])):
            numbers += Arr([0,0,1,1,0,0,0,0])
    elif code == "4.3":
        emp_hr_count4_dict[filename] = len([y[y.find('ceo')+5:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('ceo')]) - \
                len([y for y in sum([x for x in results[filename] if 'уволить сотрудника' in x \
                and 'уровень игры: 4' in x], [])])
        if emp_hr_count4_dict[filename] < min(3, emp_hr_count3_dict[filename]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "4.4":
        comp_x = [comp for comp, names in list(comp_cr2_dict.items()) + list(comp_cr3_dict.items()) + list(comp_cr4_dict.items()) \
                          if filename in names]
        balances = []
        for comp in comp_x[:]:
            balance_x = 0
            index_max_x = max([results[filename].index(x) for x in results[filename] if 'нанять сотрудника' in x \
                            and 'отклонена' in x and 'компания: '+comp in x], default=0)
            for x in results[filename][:index_max_x+1]:
                if 'выполнена' in x and 'компани' in ' '.join(x) and comp in ' '.join(x):
                    if 'перевести талеры на счет компании' in x and 'собственник: '+filename[7:-5].lower() in x:
                        cost_raw = [y for y in x if y.startswith('взнос')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'нанять сотрудника' in x and 'ceo: '+filename[7:-5].lower():
                        cost_raw = [y for y in x if y.startswith('зарплата')][0]
                        balance_x -= int(cost_raw[cost_raw.find(' ')+1 :])
                    elif 'покупка готовой продукции' in x or 'заключить сделки' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        num_raw = [y for y in x if y.startswith('кол-во')][0]
                        if 'получатель компания: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                        elif 'ceo: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                    elif 'купить продукт B2C' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        if 'у компании: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                        elif 'покупатель: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'инвестировать' in x:
                        cost_raw = [y for y in x if y.startswith('инвестиции')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
            balances.append(balance_x)
        if max(balances, default=0) >= 50 and emp_hr_count3_dict[filename] < min(3, emp_hr_count2_dict[filename]):
            numbers += Arr([0,0,-1,0,0,0,0,0])
    elif code == "4.5":
        comp_x = [comp for comp, names in list(comp_cr2_dict.items()) + list(comp_cr3_dict.items()) + list(comp_cr4_dict.items()) \
                          if filename in names]
        balances = []
        for comp in comp_x[:]:
            balance_x = 0
            index_max_x = max([results[filename].index(x) for x in results[filename] if 'нанять сотрудника' in x \
                            and 'компания: '+comp in x], default=0)
            for x in results[filename][:index_max_x+1]:
                if 'выполнена' in x and 'компани' in ' '.join(x) and comp in ' '.join(x):
                    if 'перевести талеры на счет компании' in x and 'собственник: '+filename[7:-5].lower() in x:
                        cost_raw = [y for y in x if y.startswith('взнос')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'нанять сотрудника' in x and 'ceo: '+filename[7:-5].lower():
                        cost_raw = [y for y in x if y.startswith('зарплата')][0]
                        balance_x -= int(cost_raw[cost_raw.find(' ')+1 :])
                    elif 'покупка готовой продукции' in x or 'заключить сделки' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        num_raw = [y for y in x if y.startswith('кол-во')][0]
                        if 'получатель компания: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                        elif 'ceo: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                    elif 'купить продукт B2C' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        if 'у компании: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                        elif 'покупатель: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'инвестировать' in x:
                        cost_raw = [y for y in x if y.startswith('инвестиции')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
            balances.append(balance_x)
        if max(balances, default=0) >= 50 and emp_hr_count3_dict[filename] < min(3, emp_hr_count2_dict[filename]):
            numbers += Arr([0,1,0,0,0,0,0,0])
    elif code == "4.6":
        comp_x = [comp for comp, names in list(comp_cr2_dict.items()) + list(comp_cr4_dict.items()) if filename in names]
        if any([comp in buy_rnd4_list and buy_rnd4_list.count(comp) >1 for comp in comp_x]):
            numbers += Arr([1,0,1,0,0,0,0,0])
    elif code == "4.7":
        if any([y[y.find('от компании получен  продукт(услуга)')+38:]  in buy_rnd4_list \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and 'уровень игры: 4' in x and 'ceo: {}'.format(filename[7:-5]).lower() not in x], []) \
                if y.startswith('от компании получен  продукт(услуга)')]):
            numbers += Arr([1,0,0,0,0,0,0,0])
    elif code == "4.8":
        if ('продать компанию', '4') in tables[filename]:
            numbers += Arr([2,2,1,2,2,0,0,0])
    elif code == "4.9":
        if ('продать компанию', '4') in tables[filename] \
                and ('создать компанию', '4') in tables[filename][tables[filename].index(('продать компанию', '4')):]:
            numbers += Arr([1,0,0,0,1,0,0,0])
    elif code == "4.10":
        if ('продать компанию', '4') in tables[filename] \
                and any([y[y.find('инженер')+9:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('инженер')]):
            numbers += Arr([-1,-1,-1,-1,-1,0,0,0])
    elif code == "4.11":
        comp_cr_total_x = [y[y.find('название')+10:] \
                for y in sum([x for x in results[filename] if 'создать компанию' in x], []) if y.startswith('название')]
        if len(comp_cr_total_x) > 1:
            if all([y[y.find(':')+2:] in comp_cr_total_x \
                   for y in sum([x for x in results[filename] if 'заключить сделки' in x], []) \
                   if any([y.startswith('от компании'), y.startswith('получатель')])]):
                numbers += Arr([0,0,0,0,0,0,4,0])
    elif code == "4.12":
        if any([y[y.find('покупатель')+12:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'купить продукт B2C' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('покупатель')]) \
                and any([y[y.find('покупатель')+12:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'купить продукт B2C' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('покупатель')]):
            numbers += Arr([0,1,0,1,0,0,0,0])
    elif code == "4.13":
        if any([y[y.find('ceo')+5:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and 'уровень игры: 4' not in x], []) if y.startswith('ceo')]) \
                and any([y[y.find('покупатель')+12:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'купить продукт B2C' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('покупатель')]):
            numbers += Arr([0,0,0,1,1,0,0,0])
    elif code == "4.14":
        if any([y[y.find('ceo')+5:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'покупка готовой продукции' in x \
                and 'уровень игры: 4' not in x], []) if y.startswith('ceo')]):
            numbers += Arr([0,1,0,0,0,0,0,0])
            
    if filename in marks.keys():
        marks[filename] += numbers
    else:
        marks.update({filename: Arr([0]*8)})
        
    for key, value in marks.items():
        marks[key] = [min(y, 10) for y in value]
    
    return(1 if list(numbers) != [0]*8 else 0)

def main(filename):
    global tables, results, scores
    tables, results, scores = {}, {}, {}
    all_values, marks = {}, {}
    codes = [f"1.{i}" for i in range(1,7)] + [f"2.{i}" for i in range(10)] + [f"3.{i}" for i in range(15)] \
            + [f"4.{i}" for i in range(15)]
    try:
        tables[filename], results[filename], scores[filename] = table_result_score(filename)
    except:
        pass
    for code in codes:
        all_values[code] = []
        try:
            all_values[code].append(rules(code, filename, marks))
        except Exception as e:
            #print(e)
            all_values[code].append(0)
            marks[filename] = [0]*8
            pass

    score = [list(x)[::-1][3:]+(list(x)[::-1][:3]) for x in marks.values()][0]
    return(score, all_values)

def key_actions(filename, all_values):
    df_t = pd.DataFrame()
    key_actions = {'k1': '', 'k2': '', 'k3': '', 'k4': '', 'k5': ''}
    if all_values['1.3']:
        key_actions['k1'] = 'Проинвестировал в повышение производительности инженерного труда'
    elif all_values['1.1']:
        key_actions['k1'] = 'Получил приз по результатам конкурса инженерных решений'
    elif all_values['1.4']:
        key_actions['k1'] = 'Успешно идентифицировал возможность заработка в качестве сотрудника корпорации'
    elif filename in sum(comp_cr2_dict.values(), []):
        key_actions['k1'] = 'При первой возможности перешёл в предпринимательскую позицию'
    else:
        key_actions['k1'] = 'Пропуск'

    if all_values['2.7']:
        key_actions['k2'] = 'Прошёл квалификацию на роль поставщика крупной корпорации'
    elif tables[filename].count(('заключить сделки', '3')) >2 and not all_values['1.4']:
        key_actions['k2'] = 'Смог определить рамки и сконструировать деятельность компании' \
        'в условиях высокой неопределённости и отсутствия внешнего заказчика'
    elif filename in sum(comp_cr2_dict.values(), []):
        key_actions['k2'] = 'При первой возможности перешёл в предпринимательскую позицию'
    elif all_values['1.4']:
        key_actions['k2'] = 'Успешно освоил рамки поставленной старшим партнёром позиции'
    else:
        key_actions['k2'] = 'Пропуск'


    if tables[filename].count(('заключить сделки', '3')) >2 and not all_values['1.4'] \
            and key_actions['k2'] != 'Смог определить рамки и сконструировать деятельность компании' \
            'в условиях высокой неопределённости и отсутствия внешнего заказчика':
        key_actions['k3'] = 'Смог определить рамки и сконструировать деятельность компании' \
        'в условиях высокой неопределённости и отсутствия внешнего заказчика'
    elif any([y[y.find('инвестор')+10:] not in filename.lower() \
            for y in sum([x for x in results[filename] if 'инвестировать' in x], []) if y.startswith('инвестор')]):
        key_actions['k3'] = 'Успешно провёл переговоры с инвестором и получил инвестиции'
    elif all_values['1.4'] and len([x for x in results[filename] if 'заключить сделки' in x \
            and 'тип договора: продажа инструмента' not in x and 'уровень игры: 3' in x]) >1:
        key_actions['k3'] = 'Выполнил задание старшего партнёра на строительство производства'
    else:
        key_actions['k3'] = 'Пропуск'

    if all_values['4.14']:
        key_actions['k4'] = 'Организовал производство крупной партии продукции по заказу старшего партнёра'
    elif all_values['4.7']:
        key_actions['k4'] = 'Корректно идентифицировал и занял нишу в системе разделения труда'
    elif any([y[y.find('инвестор')+10:] not in filename.lower() \
            for y in sum([x for x in results[filename] if 'инвестировать' in x], []) if y.startswith('инвестор')]) \
            and key_actions['k3'] != 'Успешно провёл переговоры с инвестором и получил инвестиции':
        key_actions['k4'] = 'Успешно провёл переговоры с инвестором и получил инвестиции'
    elif all_values['4.8']:
        key_actions['k4'] = 'Продал построенную компанию'
    else:
        key_actions['k4'] = 'Пропуск'

    if all_values['4.8'] and key_actions['k4']!= 'Продал построенную компанию':
        key_actions['k5'] ='Продал построенную компанию'
    elif ('найм в качестве директора компании', '4') in tables[filename] and ('продать компанию', '4') in tables[filename] \
            and ('заключить сделки', '4') in tables[filename][tables[filename].index(('продать компанию', '4')):]:
        key_actions['k5'] ='После продажи компании принял предложение о работе наёмным директором \
                и подтвердил эффективность проданной компании'
    elif all_values['4.9']:
        key_actions['k5'] ='После продажи компании открыл новую компанию'
    else:
        key_actions['k5'] ='Пропуск'

    return(key_actions)

# PizzaChart

In [40]:
def radar_factory(num_vars, frame='circle'):
    
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)
    
    class RadarAxes(PolarAxes):
        name = 'radar'

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.set_theta_zero_location('N')

        def fill(self, *args, closed=True, **kwargs):
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            if x[0] != x[-1]:
                x = np.concatenate((x, [x[0]]))
                y = np.concatenate((y, [y[0]]))
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return Circle((0.5, 0.5), 0.5)
            else:
                raise ValueError("unknown value for 'frame': %s" % frame)

        def draw(self, renderer):
            if frame == 'polygon':
                gridlines = self.yaxis.get_gridlines()
                for gl in gridlines:
                    gl.get_path()._interpolation_steps = num_vars
            super().draw(renderer)

        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':
                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(num_vars))
                spine.set_transform(Affine2D().rotate(pi/8).scale(0.5).translate(.5, .5)
                                    + self.transAxes)

                return {'polar': spine}
            else:
                raise ValueError("unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta

def web1(filename, col_names, numbers, numbers_mean):
    dict1 = {col: 0 for col in col_names}
    N = 8
    theta = radar_factory(8, frame='polygon')

    angles = np.linspace(0, 2 * pi, N, endpoint=False)
    angles_mids = angles + (angles[1] / 2)

    fig, ax = plt.subplots(figsize=(11, 8), subplot_kw=dict(projection='radar'))
    ax.xaxis.set_minor_locator(FixedLocator(angles))
    fig.subplots_adjust(top=0.95, bottom=0.05, left=0.07, right=0.85)
    plt.ylim([0, 10])

    ax.set_theta_offset(pi/8-pi/4)
    ax.set_theta_direction(1)
    ax.set_xticks(angles_mids)
    ax.set_xticklabels(dict1.keys())
    ax.grid(True, axis='x', which='minor')
    ax.grid(False, axis='x', which='major')
    ax.grid(True, axis='y', which='major')
    ax.set_rgrids([2,4,6,8,10], [])
    #ax.set_title(file_name,  position=(0.5, 1.1), ha='center')
    color1='springgreen'
    color2='seagreen'
    color3='tomato'
    color4='red'
    color5='yellow'
    color5='black'
    for i in range(5):
        ax.fill_between(angles[i:i+2], 0, numbers[i] , facecolor=color1, edgecolor=color2, linewidth=2)
    ax.fill_between(angles[5:7], 0, numbers[5], facecolor=color3, edgecolor=color4, linewidth=2)
    ax.fill_between(angles[6:8], 0, numbers[6], facecolor=color3, edgecolor=color4, linewidth=2)
    ax.fill_between([angles[7],(angles[7]+angles[1])], 0, numbers[7], facecolor=color3, edgecolor=color4, linewidth=2)
    
    angles_ = list(angles.copy())
    angles_.append(angles_[-1]+angles[1])
    numbers_mean_ = numbers_mean.copy()
    numbers_mean_.append(numbers_mean[0])
    numbers_mean_.append(numbers_mean[1])
    for i in range(len(numbers_mean)):
        #print(i, angles[i])
        x_values = [angles_[i], angles_[i+1]]
        y_values = [numbers_mean[i], numbers_mean[i]]
        plt.plot(x_values, y_values, ':', color='navy', linewidth=3)
        min_, max_ = min(numbers_mean_[i], numbers_mean_[i+1]), max(numbers_mean_[i], numbers_mean_[i+1])
        x_values = [angles_[i+1], angles_[i+1]]
        y_values = [min_, max_]
        plt.plot(x_values, y_values, ':', color='navy', linewidth=3)
    
    for index, label in enumerate(ax.get_xticklabels()):
        label.set_weight("bold")
        label.set_fontsize(14)
        if index in [0, 1, 7]:
            label.set_horizontalalignment("left")
        elif index in [3, 4, 5]:
            label.set_horizontalalignment("right")

    plt.savefig(filename, transparent=True)
    return(filename)

col1 = 'Дисконтирование\nустаревших\nрешений'
col2 = 'Давление на\nнезависимого\nпартнера'
col3 = 'Конструирование\n компаний'
col4 = 'Препарирование\nтехнологий'
col5 = 'Скрининг\nшансов'
col6 = 'Борьба за\nпривилегии'
col7 = 'Фиктивная\nдеятельность'
col8 = 'Отношения\nвместо дела'
col_names = [col1, col2, col3, col4, col5, col6, col7, col8]

# Мегаотчет

In [37]:
def status(filename):
    transactions = pd.read_html(filename)[0]
    if 'продать компанию' in transactions['Транзакция'].values:
        status = 'Продал компанию'
    elif 'инвестировать' in transactions['Транзакция'].values:
        status = 'Получил инвестиции'
    else:
        try:
            max_level = str(max(set(transactions['Уровень'])))
        except:
            max_level = '0'
        if max_level == '4':
            status = 'Дошёл до конца игры'
        else:
            status = 'Выбыл на {} уровне'.format(max_level)
    return(status, len(transactions))

def create_item(doc, index, profile):
    item = {}
    pizza_numbers, all_values = main(filename)
    web1(f'pizza{index}.png', col_names, pizza_numbers, profile)
    all_scores_id = drive.ListFile({'q': "title = '_сквозной_список_все_личные_счета.html' and '%s' in parents" % doc['parent_id']}).GetList()[0]['id']
    file = drive.CreateFile({'id': all_scores_id})
    file.GetContentFile('all_scores.html')
    all_scores = pd.read_html('all_scores.html')[0]
    data_dict = all_scores[all_scores['Имя'] == doc['name'][7:-5]].to_dict(orient='records')[0]
    email = data_dict['Контакт']
    item['role'] = data_dict['Роль']
    item['score'] = int(data_dict['Баланс'])
    item['gamename'] = game_name(doc['parent_id'])
    item['status'], item['transactions'] = status(filename)
    item['pizza'] = InlineImage(docx, f'pizza{index}.png', Cm(10))
    item.update(key_actions(filename, all_values))

    with open(filename, 'r', encoding='utf-8') as f:
        html = f.read()
    soup = BS(html, features='lxml')
    for image_link in soup.find_all('img'):
        if image_link['alt'] == 'фотография':
            imgURL = image_link['src']
            urllib.request.urlretrieve(imgURL, "photo.png")
            context['photo'] = InlineImage(docx, 'photo.png', Cm(10))
        if image_link['alt'] == 'карта игрока':
            folder_generated_id = drive.ListFile({'q': "title = 'generated' and '%s' in parents" % doc['parent_id']}).GetList()[0]['id']
            folder_graph_id = drive.ListFile({'q': "title = 'graph' and '%s' in parents" % folder_generated_id}).GetList()[0]['id']
            image_title = image_link['src'][16:image_link['src'].find('png')+3]
            map_id = drive.ListFile({'q': f"title = '{image_title}' and '%s' in parents" % folder_graph_id}).GetList()[0]['id']
            file = drive.CreateFile({'id': map_id})
            file.GetContentFile(f'map{index}.png')
            item['map'] = InlineImage(docx, f'map{index}.png', Cm(10))
    return(item, email)

# Статистики

In [11]:
import tqdm

#trans_n = []
#players_n = []

for gamename in tqdm.tqdm(db_list[1][1000:]):
    trans_c = 0
    print(gamename)
    game_id = drive.ListFile({'q': f"title = '{gamename}' "}).GetList()[0]['id']
    data_id = drive.ListFile({'q': "'%s' in parents" % game_id}).GetList()[0]['id']
    folder_files = drive.ListFile({'q': "title contains '_игрок_' and title contains '.html' and '%s' in parents" % data_id}).GetList()
    for file_html in folder_files[:]:
        try:
            file_html.GetContentFile('file_.html')
            file_ = pd.read_html('file_.html')[0]
            trans_c += len(file_)
        except:
            print(file_html['title'])
            pass
    trans_n.append(trans_c)
    players_n.append(len(folder_files))

NameError: name 'db_list' is not defined

# MAIN

In [12]:
drive = auth()
docs = get_docs(drive)
root_id = drive.ListFile({'q': "title = 'OBSERVER'"}).GetList()[0]['id']
nice = get_nice(drive)
docs.tail()

,name,id,parent_id
5089,_игрок_Арнольд Наталья.html,1cT3SqadvVMsLmuTOyFjgtPQkr8dkUPbq,1cbNwKQDX9xnIYgcygDT1Cfzdg5xvIZ6j
5090,_игрок_Анатолий Козлов.html,10RuUxGVXsxYdq5DnXuwA-J7qOu4QcTUY,1cbNwKQDX9xnIYgcygDT1Cfzdg5xvIZ6j
5091,_игрок_Абдулхаков Зиннур.html,1K2KQuO79bsd4QfabP16RqccmBy4NXxmt,1cbNwKQDX9xnIYgcygDT1Cfzdg5xvIZ6j
5092,_игрок_Акционер корпорации Х.html,1kVl9lJquxeHN2ftmfUetC0yyQbK-Q-Jo,1cbNwKQDX9xnIYgcygDT1Cfzdg5xvIZ6j
5093,_игрок_Starinchikova Masha.html,1GTxRVJKtS8JszZfVMyAlH792_KNatVS1,1cbNwKQDX9xnIYgcygDT1Cfzdg5xvIZ6j


In [41]:
import time
t_s=time.time()

docx = DocxTemplate("C:\\Users\\kan.mv\\Desktop\\шаблон.docx")
filename = 'file.html'
context, items, emails, phones = {}, [], [], []
table_games = pd.DataFrame(columns=['№', 'Дата и город', 'Роль', 'Cчет', 'Транзакций'])

name_1, name_2 = 'гречко','евгений'
db_list = get_db_list(drive)
docs_found = docs[docs['name'].isin(search(docs['name'], name_1, name_2))]
docs_found['gamename'] = docs_found['parent_id'].apply(game_name)
docs_found['profile'] = docs_found['gamename'].apply(lambda x: db_list[db_list['Название игры']==x]['Профиль'].values[0])
docs_found['profile'] = docs_found['profile'].apply(lambda x: [float(y) for y in x.strip('[]').split(', ')])
docs_found['order'] = docs_found['gamename'].apply(lambda x: db_list.index[db_list['Название игры'] == x][0])
docs_found.sort_values('order', inplace=True)

names = search(nice['ma_name'], name_1, name_2, True)
data_nice = nice[nice['ma_name'].isin(names)]
for row in data_nice.to_dict(orient='records'):
    emails.append(row['ma_email']+' ('+row['ma_name']+')')
    phones.append(row['phone']+' ('+row['ma_name']+')')
    
for index, doc in enumerate(docs_found.to_dict(orient='records')[:]):
    file = drive.CreateFile({'id': doc['id']})
    file.GetContentFile(filename)
    print(game_name(doc['parent_id']))
    item, email = create_item(doc, index, doc['profile'])
    items.append(item)
    if str(email) != 'nan':
        emails.append(email+' ('+doc['name'][7:-5]+')')
    table_games.loc[index] = [index+1, item['gamename'], item['role'], \
                               item['score'], item['transactions']]
        
context.update({'items': items, \
            'name': name_1.title() + ' ' + name_2.title(), \
            'emails': ', '.join(list(set(emails))), \
            'phones': ', '.join(list(set(phones))),
            'col_labels': table_games.columns, \
            'table_games': table_games.values})

docx.render(context)
docx.save(context['name']+'.docx')
print('Completed!')

print(round(time.time()-t_s))

2 марта 2019 Новосибирск
30 мая 2019 Новосибирск, Игра Сигма страт.сессия - докажи компанию
31 августа 2019 worldskills
19 октября 2019 Новосибирск
17 октября 2020 Нижний Новгород
18 декабря 2020_Турнир трех наук
25 марта 2021_АЭМ
23 апреля 2021_ТехноСпарк_Юбилей
15 июля 2021_Архипелаг
Completed!
73
